# Density Analysis
Analysis of loofa fibre density using qim3d

This notebook demonstrates a complete image analysis pipeline using the qim3d library. We'll analyse a loofah sample and carry out the following steps for analysis:
1. **Data Downloading:** Loading the scan from the qim3d repository
2. **Visualization:** Interactive exploration of the 3D structure
3. **Density Analysis:** Quantitative analysis of loofah fibre density

In [ ]:
import qim3d
from tifffile import TiffFile
from pprint import pprint
from skimage.filters import threshold_otsu
import numpy as np
import gc
from scipy.ndimage import binary_closing
from skimage.morphology import ball

# 1. Downloading and Loading the data
The qim3d data repository provides convenient access to various samples scanned at different resolutions. For computational efficiency during exploration and prototyping, downsampled versions are available alongside full-resolution samples. The `qim3d.io.Downloader` is a simple interface for listing, downloading and loading these samples.

In [ ]:
downloader = qim3d.io.Downloader()

We can list the available samples by calling `downloader.list_files()` which shows a list of all downloadable files from the QIM data repository and categorises them. 

In [ ]:
downloader.list_files()

For this task, we'll use the downsampled loofah dataset which can be downloaded by using the filename. The `load_file=True` parameter automatically loads the data into memory after downloading.

In [ ]:
data = downloader.Loofah.Loofah_DOWNSAMPLED(load_file=True)

By default, the column is loaded with the parameter `virtual_stack=True` meaning that it isn't fully read into RAM. Only it's metadata and index structure are loaded up front and portions of the volume are lazily loaded.

# 2. Visualization and Data Exploration

To get a solid understanding of the dataset, we can first read the TIFF metadata using a simple loop to display all tags:
- `BitsPerSample: 16` - The bit depth of each pixel.
- `IJMetadata` - ImageJ-specific metadata containing individual filenames for each of the slices.
- `IJMetadataByteCounts` - size in bytes for each slice.
- `Image Description` - ImageJ parameters:
    - `ImageJ=1.53a` Version of ImageJ used to create the file
    - `images=300` Total number of image slices in the stack
    - `slices=300` Number of Z-slices
    - `loop=false` Animation setting for ImageJ
    - `min=0.0, max=65535.0` Intensity range of the image data
- `ImageLength: 500` - The height of each image slice in pixels (rows).
- `ImageWidth: 500` - The width of each image slice in pixels (columns).
- `NewSubfileType: <FILETYPE.UNDEFINED: 0>` - TIFF tag indicating the type of data in the file (undefined means it's a standard image).
- `PhotometricInterpretation: <PHOTOMETRIC.MINISBLACK: 1>` - How pixel values should be interpreted (MINISBLACK means 0 = black, higher values = lighter).
- `RowsPerStrip: 500` - Number of rows stored in each data strip (matches image height, so each slice is one strip).
- `SamplesPerPixel: 1` - Number of color channels per pixel (1 = grayscale, 3 = RGB).
- `StripByteCounts: (500000,)` - Size in bytes of each data strip (500x500x2 bytes = 500,000 bytes per slice).
- `StripOffsets: (9841,)` - Byte offset where pixel data begins in the file.

In [ ]:
with TiffFile("Loofah/Loofah_DOWNSAMPLED.tif") as tif:
    meta = {tag.name: tag.value for tag in tif.pages[0].tags.values()}

pprint(meta)

qim3d offers multiple methods for visualiztaion, including interactive viewers and 3D volumetric rendering. Let's explore the dataset using the `qim3d.viz.slicer_orthogonal()` first. This viewer shows three perpendicular cross-sections (Axial (Z), Coronal (Y), Sagittal (X)), each with its own slider to adjust the slice index in real time.

In [ ]:
qim3d.viz.slicer_orthogonal(data, color_map="magma")

The `qim3d.viz.volumetric()` function provides an interactive 3D volumetric renderer that allows you to explore the internal structure of the loofah sample in three dimensions. Unlike the orthogonal slicer which shows 2D cross-sections, this volumetric renderer creates a translucent 3D representation where you can:

- **Rotate and zoom** the volume using mouse controls to examine the structure from different angles
- **Adjust opacity** to see through the outer layers and reveal internal features
- **Control transfer functions** to highlight different density ranges within the material

The volumetric renderer is particularly useful for understanding the complex 3D architecture of porous materials like loofah, where the interconnected fiber network creates intricate pathways and void spaces that are difficult to appreciate from 2D slices alone.

In [ ]:
qim3d.viz.volumetric(data)

The raw volumetric data contains a dark boundary around the loofah sample that appears as low-intensity values (below 20,000). This boundary represents imaging artifacts rather than actual material structure. We can remove these artifacts by setting all pixels below a threshold value to zero, effectively creating a cleaned dataset that focuses on the actual loofah fiber structure. This preprocessing step improves the quality of subsequent analysis and visualization by eliminating noise and artifacts from the imaging process.


In [ ]:
cleaned_data = data.copy()
cleaned_data[cleaned_data < 20000] = 0
qim3d.viz.volumetric(cleaned_data)

# 3. Image Analysis - Density Mesurement

Now that we've visualized and cleaned our loofah dataset, we'll perform quantitative analysis to measure key structural properties. In this section, we'll:

1. **Threshold the data** to separate fiber material from void spaces using Otsu's method
2. **Visualize the binary segmentation** to verify our thresholding approach
3. **Calculate fiber density**

In [ ]:
threshold_value = threshold_otsu(data)
binary_mask = data > threshold_value

In [ ]:
qim3d.viz.volumetric(binary_mask)

In [ ]:
fiber_volume = np.sum(binary_mask)
print(f"Threshold value used: {threshold_value}")
print(f"Fiber volume: {fiber_volume:,} voxels")
print(f"This represents {fiber_volume/np.prod(data.shape)*100:.1f}% of the total scan volume")

To calculate the Total Loofah Volume, we use a two-step approach:
1. **Gaussian blur** (`sigma=16`): Smooths the data and fills small gaps
2. **Morphological closing** (`ball(radius=5)`): Fills remaining holes and creates a continuous outer envelope

In [ ]:
# Step 1: Gaussian blur to smooth internal structures
print("Step 1: Applying Gaussian blur (sigma=16)...")
blurred_loofah = qim3d.filters.gaussian(cleaned_data, sigma=16)

# Step 2: Morphological closing to create solid envelope
print("Step 2: Applying morphological closing...")
structure = ball(radius=5)  # Spherical structuring element
closed_loofah = binary_closing(blurred_loofah > 0, structure=structure)

loofah_volume = np.sum(closed_loofah)

print(f"\nLoofah envelope volume: {loofah_volume:,} voxels")
print(f"This represents {loofah_volume/np.prod(data.shape)*100:.1f}% of the total scan volume")

From this, we can calculate the density of the loofah

In [ ]:
density = fiber_volume / loofah_volume
print(f"Loofah density: {density:.3f}")